# Quantitative Value Strategy
"Value investing" means investing in the stocks that are cheapest relative to common measures of business value (like earnings or assets).

For this project, we're going to build an investing strategy that selects the 50 stocks with the best value metrics. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.

## Library Imports
The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [ ]:
import numpy as np
import pandas as pd
import xlsxwriter
import requests
from scipy import stats
import math

: 

## Importing Our List of Stocks & API Token
As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the .csv file is still in your working directory and import it with the following command:

In [2]:
from secrets import IEX_CLOUD_API_TOKEN
stocks = pd.read_csv('sp_500_stocks.csv')

## Making Our First API Call
It's now time to make the first version of our value screener!

We'll start by building a simple value screener that ranks securities based on a single metric (the price-to-earnings ratio).

In [3]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
response = requests.get(api_url).json()
response

{'avgTotalVolume': 93113652,
 'calculationPrice': 'close',
 'change': -5.7,
 'changePercent': -0.04019,
 'close': 143.09,
 'closeSource': 'ciafloif',
 'closeTime': 1732149389564,
 'companyName': 'Apple Inc',
 'currency': 'USD',
 'delayedPrice': 142.39,
 'delayedPriceTime': 1724257127638,
 'extendedChange': 0.25,
 'extendedChangePercent': 0.00185,
 'extendedPrice': 140.69,
 'extendedPriceTime': 1704549116455,
 'high': 145.27,
 'highSource': 'rpeenyit uddmae5i cl1 e',
 'highTime': 1719116952653,
 'iexAskPrice': None,
 'iexAskSize': None,
 'iexBidPrice': None,
 'iexBidSize': None,
 'iexClose': 139.61,
 'iexCloseTime': 1721731858729,
 'iexLastUpdated': None,
 'iexMarketPercent': None,
 'iexOpen': 142.18,
 'iexOpenTime': 1689593002361,
 'iexRealtimePrice': None,
 'iexRealtimeSize': None,
 'iexVolume': None,
 'lastTradeTime': 1655293116908,
 'latestPrice': 140.89,
 'latestSource': 'Close',
 'latestTime': 'June 10, 2022',
 'latestUpdate': 1702313093887,
 'latestVolume': 92587015,
 'low': 141.

## Parsing Our API Call
This API call has the metric we need - the price-to-earnings ratio.

Here is an example of how to parse the metric from our API call:

In [4]:
price = response['latestPrice']
pe_ratio = response['peRatio']

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called chunks that we can use to divide our list of securities into groups of 100.

In [5]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [6]:
final_dataframe = pd.DataFrame(columns=my_columns)

for symbol_string in symbol_strings:
    batch_api_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}'
    response = requests.get(batch_api_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = pd.concat([final_dataframe,
                                    pd.DataFrame([[
                                        symbol,
                                        response[symbol]['quote']['latestPrice'],
                                        response[symbol]['quote']['peRatio'],
                                        'N/A'
                                    ]], columns = my_columns)
                                    ], ignore_index = True)
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,A,125.96,30.37,N/A
1,AAL,15.42,-4.06,N/A
2,AAP,185.19,20.24,N/A
3,AAPL,140.79,22.8,N/A
4,ABBV,144,20.98,N/A
...,...,...,...,...
498,YUM,118.29,20.93,N/A
499,ZBH,117.55,114.67,N/A
500,ZBRA,308.93,21,N/A
501,ZION,54.58,5.71,N/A


## Removing Glamour Stocks

The opposite of a "value stock" is a "glamour stock". 

Since the goal of this strategy is to identify the 50 best value stocks from our universe, our next step is to remove glamour stocks from the DataFrame.

We'll sort the DataFrame by the stocks' price-to-earnings ratio, and drop all stocks outside the top 50.

In [7]:
final_dataframe.sort_values('Price-to-Earnings Ratio', inplace=True)
final_dataframe = final_dataframe[final_dataframe['Price-to-Earnings Ratio'] > 0]
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(drop=True, inplace=True)

In [8]:
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,EBAY,46.3,2.51,N/A
1,AMZN,109.89,2.72,N/A
2,NRG,45.53,2.78,N/A
3,MRNA,132.72,3.88,N/A
4,SYF,33.04,4.27,N/A
5,NUE,123.89,4.35,N/A
6,PARA,28.59,4.59,N/A
7,COF,117.72,4.6,N/A
8,F,12.76,4.62,N/A
9,AIG,54.04,4.68,N/A


## Calculating the Number of Shares to Buy
We now need to calculate the number of shares we need to buy. 

To do this, we will use the `portfolio_input` function that we created in our momentum project.

I have included this function below.

In [9]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

Use the `portfolio_input` function to accept a `portfolio_size` variable from the user of this script.

In [10]:
portfolio_input()

You can now use the global `portfolio_size` variable to calculate the number of shares that our strategy should purchase.

In [11]:
position_size = float(portfolio_size)/len(final_dataframe)
for row in final_dataframe.index:
    final_dataframe.loc[row, 'Number of Shares to Buy'] = math.floor(position_size/final_dataframe.loc[row, 'Price'])

final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,EBAY,46.3,2.51,4319
1,AMZN,109.89,2.72,1820
2,NRG,45.53,2.78,4392
3,MRNA,132.72,3.88,1506
4,SYF,33.04,4.27,6053
5,NUE,123.89,4.35,1614
6,PARA,28.59,4.59,6995
7,COF,117.72,4.6,1698
8,F,12.76,4.62,15673
9,AIG,54.04,4.68,3700


## Building a Better (and More Realistic) Value Strategy
Every valuation metric has certain flaws.

For example, the price-to-earnings ratio doesn't work well with stocks with negative earnings.

Similarly, stocks that buyback their own shares are difficult to value using the price-to-book ratio.

Investors typically use a `composite` basket of valuation metrics to build robust quantitative value strategies. In this section, we will filter for stocks with the lowest percentiles on the following metrics:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
* Enterprise Value divided by Gross Profit (EV/GP)

Some of these metrics aren't provided directly by the IEX Cloud API, and must be computed after pulling raw data. We'll start by calculating each data point from scratch.

In [12]:
symbol = 'AAPL'
batch_api_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
response = requests.get(batch_api_url).json()
print(response[symbol]['advanced-stats'])

# Price-to-earnings ratio
pe_ratio = response[symbol]['quote']['peRatio']


# Price-to-book ratio
pb_ratio = response[symbol]['advanced-stats']['priceToBook']


# Price-to-sales ratio
ps_ratio = response[symbol]['advanced-stats']['priceToSales']


# Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
enterprise_value = response[symbol]['advanced-stats']['enterpriseValue']
ebitda = response[symbol]['advanced-stats']['EBITDA']
ev_to_ebitda = enterprise_value/ebitda
print(ev_to_ebitda)



# Enterprise Value divided by Gross Profit (EV/GP)
enterprise_value = response[symbol]['advanced-stats']['enterpriseValue']
gross_profit = response[symbol]['advanced-stats']['grossProfit']
ev_to_gross_profit = enterprise_value/gross_profit
print(ev_to_gross_profit)



{'beta': 1.327761487308146, 'totalCash': 52766867328, 'currentDebt': 131147030303, 'revenue': 395901248915, 'grossProfit': 170147920120, 'totalRevenue': 402237190196, 'EBITDA': 136553271689, 'revenuePerShare': 24.18, 'revenuePerEmployee': 2715433.65, 'debtToEquity': 5.335426245815926, 'profitMargin': 0.2644962427604887, 'enterpriseValue': 2323513974751, 'enterpriseValueToRevenue': 5.97, 'priceToSales': 5.86, 'priceToBook': 34.17, 'forwardPERatio': 23.1520923408795, 'pegRatio': 0.5751276162923008, 'peHigh': 13.128100279144144, 'peLow': 7.03200366652134, 'week52highDate': '2021-12-21', 'week52lowDate': '2021-06-11', 'putCallRatio': 0.5312868942413107, 'companyName': 'Apple Inc', 'marketcap': 2299117369042, 'week52high': 187.19, 'week52low': 132.5, 'week52highSplitAdjustOnly': 188.21, 'week52highDateSplitAdjustOnly': '2021-12-21', 'week52lowSplitAdjustOnly': 131.6, 'week52lowDateSplitAdjustOnly': '2021-06-02', 'week52change': 0.08296614109645666, 'sharesOutstanding': 16748142473, 'float':

Let's move on to building our DataFrame. You'll notice that I use the abbreviation `rv` often. It stands for `robust value`, which is what we'll call this sophisticated strategy moving forward.

In [13]:
rv_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'Price-to-Earnings Ratio',
    'PE Percentile',
    'Price-to-Book Ratio',
    'PB Percentile',
    'Price-to-Sales Ratio',
    'PS Percentile',
    'EV/EBITDA',
    'EV/EBITDA Percentile',
    'EV/GP',
    'EV/GP Percentile',
    'RV Score'
]

rv_dataframe = pd.DataFrame(columns=rv_columns)

for symbol_string in symbol_strings:
    batch_api_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
    response = requests.get(batch_api_url).json()
    for symbol in symbol_string.split(','):

        enterprise_value = response[symbol]['advanced-stats']['enterpriseValue']
        ebitda = response[symbol]['advanced-stats']['EBITDA']
        try:
            ev_to_ebitda = enterprise_value/ebitda
        except TypeError:
            ev_to_ebitda = np.NaN
        
        gross_profit = response[symbol]['advanced-stats']['grossProfit']
        try:
            ev_to_gross_profit = enterprise_value/gross_profit
        except TypeError:
            ev_to_gross_profit = np.NaN
        
        rv_dataframe = pd.concat([
                                rv_dataframe,
                                pd.DataFrame([[
                                    symbol,
                                    response[symbol]['quote']['latestPrice'],
                                    'N/A',
                                    response[symbol]['quote']['peRatio'],
                                    'N/A',
                                    response[symbol]['advanced-stats']['priceToBook'],
                                    'N/A',
                                    response[symbol]['advanced-stats']['priceToSales'],
                                    'N/A',
                                    ev_to_ebitda,
                                    'N/A',
                                    ev_to_gross_profit,
                                    'N/A',
                                    'N/A'
                                    
                                ]],
                                columns=rv_columns)
                                
                        ], ignore_index=True)


In [14]:
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,122.61,N/A,30.07,N/A,7.21,N/A,5.77,N/A,20.617019,N/A,10.492884,N/A,N/A
1,AAL,15.1,N/A,-4.19,N/A,-1.1,N/A,0.2802,N/A,-22.179974,N/A,0.987515,N/A,N/A
2,AAP,186.52,N/A,20.02,N/A,3.81,N/A,0.9859,N/A,11.044086,N/A,2.325856,N/A,N/A
3,AAPL,138.45,N/A,22.6,N/A,34.49,N/A,5.85,N/A,17.929871,N/A,13.551275,N/A,N/A
4,ABBV,146.9,N/A,21.07,N/A,15.61,N/A,4.6,N/A,10.903524,N/A,7.964115,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
498,YUM,117.66,N/A,21.07,N/A,-4,N/A,5.03,N/A,19.733315,N/A,9.101885,N/A,N/A
499,ZBH,115.37,N/A,114.22,N/A,1.99,N/A,3.12,N/A,18.535544,N/A,5.718648,N/A,N/A
500,ZBRA,314.53,N/A,21,N/A,5.65,N/A,2.92,N/A,16.102346,N/A,6.730874,N/A,N/A
501,ZION,52.79,N/A,5.5,N/A,1.38,N/A,2.38,N/A,4.428355,N/A,2.168569,N/A,N/A


## Dealing With Missing Data in Our DataFrame

Our DataFrame contains some missing data because all of the metrics we require are not available through the API we're using. 

You can use pandas' `isnull` method to identify missing data:

In [15]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
38,AON,261.02,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
55,BALL,70.01,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
70,BRK.B,317.5,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
191,FOX,31.03,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
194,FRT,103.12,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
205,GOOG,2296.66,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
318,MTB,172.99,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
342,NWS,16.98,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
443,TROW,120.19,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
454,UA,9.68,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A


Dealing with missing data is an important topic in data science.

There are two main approaches:

* Drop missing data from the data set (pandas' `dropna` method is useful here)
* Replace missing data with a new value (pandas' `fillna` method is useful here)

In this tutorial, we will replace missing data with the average non-`NaN` data point from that column. 

Here is the code to do this:

In [16]:
for column in ['Price-to-Earnings Ratio', 'Price-to-Book Ratio', 'Price-to-Sales Ratio', 'EV/EBITDA', 'EV/GP']:
    rv_dataframe[column].fillna(rv_dataframe[column].mean(), inplace=True)

Now, if we run the statement from earlier to print rows that contain missing data, nothing should be returned:

In [17]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score


## Calculating Value Percentiles

We now need to calculate value score percentiles for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* EV/EBITDA
* EV/GP

Here's how we'll do this:

In [46]:
from scipy.stats import percentileofscore as score

metrics = {
    'Price-to-Earnings Ratio': 'PE Percentile',
    'Price-to-Book Ratio': 'PB Percentile',
    'Price-to-Sales Ratio': 'PS Percentile',
    'EV/EBITDA': 'EV/EBITDA Percentile',
    'EV/GP': 'EV/GP Percentile'
}

for metric in metrics.keys():
    for row in rv_dataframe.index:
        rv_dataframe.loc[row, metrics[metric]] = score(rv_dataframe[metric], rv_dataframe.loc[row, metric])/100

rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,AAL,15.1,13245,-4.19,0.06,-1.1000,0.12,0.2802,0.04,-22.179974,0.04,0.987515,0.22,2.703777
1,AIG,55.64,3594,4.89,0.2,0.7757,0.22,0.8106,0.54,2.875157,0.14,0.821491,0.14,5.606362
2,PVH,65.68,3045,5.06,0.22,0.8655,0.3,0.5000,0.22,4.611980,0.44,1.027015,0.28,5.646123
3,GM,36.26,5515,6.06,0.44,0.8401,0.26,0.3967,0.12,1.937168,0.06,1.620347,0.48,5.685885
4,LNC,53.46,3741,7.59,0.62,0.6111,0.18,0.4727,0.18,5.385439,0.6,0.461363,0.02,6.282306
5,PRU,99.2,2016,7.90,0.66,0.8760,0.32,0.5662,0.24,5.199994,0.54,0.555760,0.04,6.878728
6,F,13.35,14981,4.60,0.18,1.1400,0.56,0.3810,0.08,4.095832,0.32,2.067051,0.62,7.157058
7,SYF,32.42,6169,4.28,0.1,1.2700,0.62,0.9398,0.66,2.716802,0.12,1.000262,0.26,7.455268
8,MET,66.37,3013,8.31,0.68,0.9755,0.46,0.7468,0.46,4.740571,0.5,0.730347,0.1,8.151093
9,C,50.05,3996,5.78,0.36,0.5383,0.16,1.2100,0.8,3.499406,0.22,1.138751,0.38,8.210736


## Calculating the RV Score
We'll now calculate our RV Score (which stands for Robust Value), which is the value score that we'll use to filter for stocks in this investing strategy.

The RV Score will be the arithmetic mean of the 4 percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the mean function from Python's built-in statistics module.

In [47]:
from statistics import mean
for row in rv_dataframe.index:
    rv_dataframe.loc[row, 'RV Score'] = mean([
        rv_dataframe.loc[row, 'PE Percentile'],
        rv_dataframe.loc[row, 'PB Percentile'],
        rv_dataframe.loc[row, 'PS Percentile'],
        rv_dataframe.loc[row, 'EV/EBITDA Percentile'],
        rv_dataframe.loc[row, 'EV/GP Percentile'],]

    )

## Selecting the 50 Best Value Stocks¶

As before, we can identify the 50 best value stocks in our universe by sorting the DataFrame on the RV Score column and dropping all but the top 50 entries.

In [48]:
rv_dataframe.sort_values('RV Score', ascending=True, inplace=True)
rv_dataframe = rv_dataframe[:50]
rv_dataframe.reset_index(drop=True, inplace=True)

/var/folders/c_/9cqyc6w948j02tgk3jpzkpvc0000gn/T/ipykernel_6391/4064645911.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rv_dataframe.sort_values('RV Score', ascending=True, inplace=True)


In [49]:
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,AAL,15.1,13245,-4.19,0.06,-1.1000,0.12,0.2802,0.04,-22.179974,0.04,0.987515,0.22,0.096
1,AIG,55.64,3594,4.89,0.2,0.7757,0.22,0.8106,0.54,2.875157,0.14,0.821491,0.14,0.248
2,GM,36.26,5515,6.06,0.44,0.8401,0.26,0.3967,0.12,1.937168,0.06,1.620347,0.48,0.272
3,UAL,42.9,4662,-6.80,0.04,3.8000,1.0,0.4672,0.16,-51.111932,0.02,0.933446,0.2,0.284
4,PVH,65.68,3045,5.06,0.22,0.8655,0.3,0.5000,0.22,4.611980,0.44,1.027015,0.28,0.292
5,LNC,53.46,3741,7.59,0.62,0.6111,0.18,0.4727,0.18,5.385439,0.6,0.461363,0.02,0.32
6,CAH,56.07,3566,-16.45,0.02,-21.0000,0.06,0.0846,0.02,8.195624,0.92,2.715010,0.72,0.348
7,SYF,32.42,6169,4.28,0.1,1.2700,0.62,0.9398,0.66,2.716802,0.12,1.000262,0.26,0.352
8,F,13.35,14981,4.60,0.18,1.1400,0.56,0.3810,0.08,4.095832,0.32,2.067051,0.62,0.352
9,PRU,99.2,2016,7.90,0.66,0.8760,0.32,0.5662,0.24,5.199994,0.54,0.555760,0.04,0.36


## Calculating the Number of Shares to Buy
We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a for loop to calculate the number of shares to buy for each stock in our investment universe.

In [50]:
portfolio_input()

In [51]:
position_size = float(portfolio_size)/len(rv_dataframe)
for row in rv_dataframe.index:
    rv_dataframe.loc[row, 'Number of Shares to Buy'] = math.floor(position_size/rv_dataframe.loc[row, 'Price'])

In [52]:
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,AAL,15.1,13245,-4.19,0.06,-1.1000,0.12,0.2802,0.04,-22.179974,0.04,0.987515,0.22,0.096
1,AIG,55.64,3594,4.89,0.2,0.7757,0.22,0.8106,0.54,2.875157,0.14,0.821491,0.14,0.248
2,GM,36.26,5515,6.06,0.44,0.8401,0.26,0.3967,0.12,1.937168,0.06,1.620347,0.48,0.272
3,UAL,42.9,4662,-6.80,0.04,3.8000,1.0,0.4672,0.16,-51.111932,0.02,0.933446,0.2,0.284
4,PVH,65.68,3045,5.06,0.22,0.8655,0.3,0.5000,0.22,4.611980,0.44,1.027015,0.28,0.292
5,LNC,53.46,3741,7.59,0.62,0.6111,0.18,0.4727,0.18,5.385439,0.6,0.461363,0.02,0.32
6,CAH,56.07,3566,-16.45,0.02,-21.0000,0.06,0.0846,0.02,8.195624,0.92,2.715010,0.72,0.348
7,SYF,32.42,6169,4.28,0.1,1.2700,0.62,0.9398,0.66,2.716802,0.12,1.000262,0.26,0.352
8,F,13.35,14981,4.60,0.18,1.1400,0.56,0.3810,0.08,4.095832,0.32,2.067051,0.62,0.352
9,PRU,99.2,2016,7.90,0.66,0.8760,0.32,0.5662,0.24,5.199994,0.54,0.555760,0.04,0.36


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [61]:
writer = pd.ExcelWriter('Value_Strategy.xlsx', engine='xlsxwriter')
rv_dataframe.to_excel(writer, sheet_name='Value Strategy', index=False)

## Creating the Formats We'll Need For Our .xlsx File
You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase
* Float formats with 1 decimal for each valuation metric

Since we already built some formats in past sections of this course, I've included them below for you. Run this code cell before proceeding.

In [62]:
background_color = '#0a0a23'
font_color = '#ffffff'
header_background_color = '#fffdd0'
header_font_color = '#000000'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1,
            'font_size': 14,
            'align': 'left'
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1,
            'font_size': 14,
            'align': 'left'
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1,
            'font_size': 14,
            'align': 'left'
        }
    )

float_template = writer.book.add_format(
    {
        'num_format' : '0.0',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1,
        'font_size': 14,
        'align': 'left'
    }
)

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1,
            'font_size': 14,
            'align': 'left'
        }
    )

header_template = writer.book.add_format(
        {
            'font_color': header_font_color,
            'bg_color': header_background_color,
            'border': 1,
            'font_size': 16
        }
    )

In [63]:
column_formats = {
    'A': ['Ticker', string_template],
    'B': ['Price', dollar_template],
    'C': ['Number of Shares to Buy', integer_template],
    'D': ['Price-to-Earnings Ratio', float_template],
    'E': ['PE Percentile', percent_template],
    'F': ['Price-to-Book Ratio', float_template],
    'G': ['PB Percentile', percent_template],
    'H': ['Price-to-Sales Ratio', float_template],
    'I': ['PS Percentile', percent_template],
    'J': ['EV/EBITDA', float_template],
    'K': ['EV/EBITDA Percentile', percent_template],
    'L': ['EV/GP', float_template],
    'M': ['EV/GP Percentile', percent_template],
    'N': ['RV Score', string_template]
}

for column in column_formats:
    writer.sheets['Value Strategy'].set_column(f'{column}:{column}', 30, column_formats[column][1])
    writer.sheets['Value Strategy'].write(f'{column}1', column_formats[column][0], header_template)

## Saving Our Excel Output
As before, saving our Excel output is very easy:

In [64]:
writer.save()